In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.utils import image_utils
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
# Set root directory
root_dir = 'spectrogram/'

# Define the image classes
classes = os.listdir(root_dir)

# Set the data generator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
batch_size = 10

# Load the images using the data generator
train_generator = datagen.flow_from_directory(
    root_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    root_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Save the class labels
class_labels = list(train_generator.class_indices.keys())

# Convert the generators to numpy arrays
train_images, train_labels = next(train_generator)
test_images, test_labels = next(validation_generator)

Found 17585 images belonging to 20 classes.
Found 1943 images belonging to 20 classes.


In [4]:
print(train_images[0].shape)

(128, 128, 3)


# Buidling Model

In [5]:
model = tf.keras.models.Sequential()

## Conv Layer 1

In [6]:
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[128, 128, 3]))

In [7]:
model.add(tf.keras.layers.BatchNormalization())

In [8]:
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [9]:
model.add(tf.keras.layers.Dropout(0.25))

## Conv Layer 2

In [10]:
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))

In [11]:
model.add(tf.keras.layers.BatchNormalization())

In [12]:
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [13]:
model.add(tf.keras.layers.Dropout(0.25))

## Conv Layer 3

In [14]:
model.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3,activation='relu'))

In [15]:
model.add(tf.keras.layers.BatchNormalization())

In [16]:
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [17]:
model.add(tf.keras.layers.Dropout(0.25))

## Conv Layer 4

In [18]:
model.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,activation='relu'))

In [19]:
model.add(tf.keras.layers.BatchNormalization())

In [20]:
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [21]:
model.add(tf.keras.layers.Dropout(0.25))

## Flattening

In [22]:
model.add(tf.keras.layers.Flatten())

## Dense Layer 1

In [23]:
model.add(tf.keras.layers.Dense(64))

In [24]:
model.add(tf.keras.layers.BatchNormalization())

In [25]:
model.add(tf.keras.layers.Dropout(0.25))

## Dense Layer 2

In [26]:
model.add(tf.keras.layers.Dense(32))

In [27]:
model.add(tf.keras.layers.BatchNormalization())

In [28]:
model.add(tf.keras.layers.Dropout(0.25))

## Output Layer

In [29]:
model.add(tf.keras.layers.Dense(20,activation='softmax'))

In [30]:
model.compile(optimizer = 'adam', loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 126, 126, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 63, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 61, 61, 64)       2

In [32]:
model.fit(train_generator, epochs=10, validation_data=validation_generator)

Epoch 1/10
1759/1759 [==============================] - 72s 39ms/step - loss: 1.9379 - accuracy: 0.4525 - val_loss: 1.6019 - val_accuracy: 0.5286
Epoch 2/10
1759/1759 [==============================] - 67s 38ms/step - loss: 1.4636 - accuracy: 0.5781 - val_loss: 1.5103 - val_accuracy: 0.5543
Epoch 3/10
1759/1759 [==============================] - 68s 39ms/step - loss: 1.3056 - accuracy: 0.6213 - val_loss: 1.4223 - val_accuracy: 0.5980
Epoch 4/10
1759/1759 [==============================] - 69s 39ms/step - loss: 1.1917 - accuracy: 0.6533 - val_loss: 1.4261 - val_accuracy: 0.5955
Epoch 5/10
1759/1759 [==============================] - 67s 38ms/step - loss: 1.0946 - accuracy: 0.6799 - val_loss: 1.4094 - val_accuracy: 0.6011
Epoch 6/10
1759/1759 [==============================] - 71s 41ms/step - loss: 1.0386 - accuracy: 0.6984 - val_loss: 1.3952 - val_accuracy: 0.6058
Epoch 7/10
1759/1759 [==============================] - 69s 39ms/step - loss: 0.9707 - accuracy: 0.7129 - val_loss: 1.3762 -

In [35]:
model.save('bird_voice.h5')